In [1]:
import numpy as np
import pandas as pd
import os

# Helper Functions

In [2]:
def output_data(file, algorithm):
    data = pd.read_csv(file, sep=" ", header=None, error_bad_lines=False)
    if data.shape[1] == 5:
        data.drop(data.columns[len(data.columns) - 1], axis=1, inplace=True)
    data.columns = ["k", "s", "cost", "file"]
    data['algorithm'] = algorithm
    return data


def count_files(directory, input_size, extension):
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith(str(input_size) + str(extension)):
            count += 1
    return count

## Merge Outputs

In [3]:
import shutil
import os

def copy_outputs(df, output_dir_old, output_dir_new):
    if not os.path.exists(output_dir_new):
        os.makedirs(output_dir_new)
    for file in df['file'].tolist():
        file_code = file.split("/")[1].split('.in')[0]
        shutil.copy2(str(output_dir_old) + str(file_code) + '.out', str(output_dir_new))
        
def merge_best_outputs(ant_file, tsp_file, ant_out_dir, tsp_out_dir, final_out_dir):
    if ant_file and tsp_file:
        ant_data = output_data(ant_file, 'ant')
        tsp_data = output_data(tsp_file, 'tsp')
        merged_tsp_ant = pd.merge(ant_data, tsp_data, left_on='file', right_on='file', suffixes=('_ant', '_tsp'))
        ant_better = merged_tsp_ant[merged_tsp_ant['cost_ant'] < merged_tsp_ant['cost_tsp']]
        tsp_better = merged_tsp_ant[merged_tsp_ant['cost_ant'] >= merged_tsp_ant['cost_tsp']]
        copy_outputs(ant_better, ant_out_dir, final_out_dir)
        copy_outputs(tsp_better, tsp_out_dir, final_out_dir)
    elif tsp_file and not ant_file:
        tsp_data = output_data(tsp_file, 'tsp')
        copy_outputs(tsp_data, tsp_out_dir, final_out_dir)

In [4]:
in_50 = count_files('inputs', '50', '.in')
in_100 = count_files('inputs', '100', '.in')
in_200 = count_files('inputs', '200', '.in')
in_50, in_100, in_200

(318, 316, 315)

In [5]:
tsp_50 = count_files('outputs_tsp_50', '50', '.out')
tsp_100 = count_files('outputs_tsp_100', '100', '.out')
tsp_200 = count_files('outputs_tsp_200_old', '200', '.out')
tsp_50, tsp_100, tsp_200

(318, 316, 194)

In [6]:
ant_50 = count_files('outputs_ant_50', '50', '.out')
ant_100 = count_files('outputs_ant_100', '100', '.out')
ant_50, ant_100

(318, 316)

In [7]:
inputs = set(os.listdir('inputs'))
outputs_200 = set(os.listdir('outputs_tsp_200_old'))
missing_outputs_200 = []
for filename in inputs:
    file = filename.split(".")[0]
    if '_200' in file and file + '.out' not in outputs_200:
        missing_outputs_200.append(filename)

In [8]:
len(missing_outputs_200)

121

In [ ]:
missing_outputs_200

In [ ]:
from solver import solve_from_file

for missing_input_file in missing_outputs_200:   
    solve_from_file('inputs/' + missing_input_file, 'outputs_tsp_200', params=["TSP", "200"])